Задание:

На основе Правил безопасности опасных производственных объектов, на которых используются подъемные сооружения (Документ для создания базы знаний: https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc), сделайте:
- нейро-консультанта для ответов на вопросы по представленному документу, проработайте промт самостоятельно;
- Проверьте работоспособность созданного консультанта на нескольких самостоятельно сформулированных вопросах.

In [ ]:
# установка библиотек

!pip install faiss-cpu==1.8.0 openai==1.55.3 tiktoken==0.7.0 langchain_openai==0.1.22 langchain_community==0.2.12

In [ ]:
# импорт библиотек

import os
import getpass
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [ ]:
# ключ от OpenAI

openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

In [ ]:
# Создание клиента OpenAI с использованием API ключа из переменных среды





In [ ]:
# функция для загрузки документа по ссылке из гугл драйв

def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc')
data_from_url

In [ ]:
# деление на чанки
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)

for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={}))

source_chunks

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

In [ ]:
# основная функция для получения ответов по БЗ

def answer_index(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией для ответа клиенту в ответе. Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0
    )
    answer = completion.choices[0].message.content
    return answer

In [ ]:
# самостоятельно напишите промпт
system="Ты – консультант на производственном объекте, ответь на вопрос сотрудника строго на основе предоставленной информации. Не добавляй ничего от себя и не раскрывай источник информации."

In [ ]:
# проверьте работу нейро-консультанта на нескольких самостоятельно сформулированных вопросах, для получения более качественных ответов
# можно поэкспериментировать с разделением на чанки и другими параметрами


In [ ]:
# вопрос пользователя

topic= "Чем должна располагать специализированная организация?"
ans=answer_index(system, topic, db)
ans

In [ ]:
# вопрос пользователя

topic= "Что обязана делать огранизация при эксплуатации ПС?"
ans=answer_index(system, topic, db)
ans

In [ ]:
# вопрос пользователя

topic= "Какие требования нужны для установка кранов, передвигающихся по надземному рельсовому пути?"
ans=answer_index(system, topic, db)
ans